segmentation 에 있는 patient 들의 segmentation image 를 만든다.

In [1]:
import os
import glob
import pydicom
import nibabel as nib
import pandas as pd
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import pylibjpeg

from tqdm import tqdm

from PIL import Image

import csv

In [2]:
DATA_DIR = "/root/autodl-tmp/cervical_spine/"
TRAIN_SAGITTAL_DIR = os.path.join(DATA_DIR, "train_sagittal_images_png")
TRAIN_CORONAL_DIR = os.path.join(DATA_DIR, "train_coronal_images_png")
TRAIN_AXIAL_DIR = os.path.join(DATA_DIR, "train_axial_images_png")
TRAIN_IMAGE_DIR = os.path.join(DATA_DIR, "train_images")

In [3]:
patient_dirs = glob.glob(TRAIN_IMAGE_DIR + "/**")
len(patient_dirs)

2012

In [4]:
patient_dirs

['/root/autodl-tmp/cervical_spine/train_images/1.2.826.0.1.3680043.10001',
 '/root/autodl-tmp/cervical_spine/train_images/1.2.826.0.1.3680043.10005',
 '/root/autodl-tmp/cervical_spine/train_images/1.2.826.0.1.3680043.10014',
 '/root/autodl-tmp/cervical_spine/train_images/1.2.826.0.1.3680043.10016',
 '/root/autodl-tmp/cervical_spine/train_images/1.2.826.0.1.3680043.10032',
 '/root/autodl-tmp/cervical_spine/train_images/1.2.826.0.1.3680043.10041',
 '/root/autodl-tmp/cervical_spine/train_images/1.2.826.0.1.3680043.10051',
 '/root/autodl-tmp/cervical_spine/train_images/1.2.826.0.1.3680043.10058',
 '/root/autodl-tmp/cervical_spine/train_images/1.2.826.0.1.3680043.10062',
 '/root/autodl-tmp/cervical_spine/train_images/1.2.826.0.1.3680043.1010',
 '/root/autodl-tmp/cervical_spine/train_images/1.2.826.0.1.3680043.10136',
 '/root/autodl-tmp/cervical_spine/train_images/1.2.826.0.1.3680043.1016',
 '/root/autodl-tmp/cervical_spine/train_images/1.2.826.0.1.3680043.10179',
 '/root/autodl-tmp/cervical

In [5]:
def rescale_img_to_hu(dcm_ds):
    """Rescales the image to Hounsfield unit.
    """
    return dcm_ds.pixel_array * dcm_ds.RescaleSlope + dcm_ds.RescaleIntercept


def read_dcm(patient_dir, num_instance):
    dcm_path = os.path.join(patient_dir, f"{int(num_instance)}.dcm")
    ds = pydicom.dcmread(dcm_path)
    img2d = rescale_img_to_hu(ds)
    return normalize_hu(img2d)

def read_patient_dcm(patient_dir):
    """
    여기서 이미지를 정상적인 순서로 돌려 놓는다
    :param patient_dir:
    :return:
    """
    num_slices = len(glob.glob(patient_dir + "/*.dcm"))
    # print(f"total slices {num_slices}")
    imgs = np.zeros((num_slices, 512, 512))
    image_positions = np.zeros((num_slices, 3))
    image_orientations = np.zeros((num_slices, 6))
    pixel_spacings = np.zeros((num_slices, 2))
    slice_thicknesses = np.zeros((num_slices, 1))

    ignore_count = 1
    for i in range(num_slices):
        dcm_path = os.path.join(patient_dir, f"{i+ignore_count}.dcm")
        while os.path.exists(dcm_path) == False:
            ignore_count += 1
            dcm_path = os.path.join(patient_dir, f"{i+ignore_count}.dcm")
        ds = pydicom.dcmread(dcm_path)

        image_positions[i, :] = ds.ImagePositionPatient
        image_orientations[i, :] = ds.ImageOrientationPatient
        pixel_spacings[i, :] = ds.PixelSpacing
        slice_thicknesses[i, :] = ds.SliceThickness

        img2d = rescale_img_to_hu(ds)

        imgs[i] = img2d

    is_flip = False
    # check z is in good direction
    if image_positions[0, 2] < image_positions[1, 2]:
        is_flip = True
        # flip image in z direction
        imgs = np.flip(imgs, axis=0)
        image_positions = np.flip(image_positions, axis=0)
        pixel_spacings = np.flip(pixel_spacings, axis=0)
        slice_thicknesses = np.flip(slice_thicknesses, axis=0)

    return imgs, image_positions, image_orientations, pixel_spacings, slice_thicknesses, is_flip


def normalize_hu(data):
    # normalize to 0-1
    # return (data - data.min()) / data.max()
    data = np.clip(data, a_min=-2242, a_max=2242) / 4484 + 0.5
    return data

def calculate_aspect(image_positions, pixel_spacings):
    """
    calculate z aspect, z 를 몇배로 늘여야 하는가야
    :param image_positions:
    :param pixel_spacings:
    :return:
    """
    height = image_positions[0, 2] - image_positions[1, 2]
    pixel_spacing = pixel_spacings[0, 0]
    aspect = height / pixel_spacing
    return aspect

In [6]:
f = open('meta_train_3d.csv','a',newline='')
writer = csv.writer(f)

In [7]:
header = [
    'UID',
    'z_spacing','pixel_spacing','aspect', 'is_flip', 'z_height'
]
writer.writerow(header)

53

In [8]:
def save_sagittal(imgs, patient_sagittal_dir, aspect):
    img_width = imgs.shape[1]
    # 그냥 int 를 취했다 round 하지 않았다.
    img_height = int(imgs.shape[0] * aspect)

    for sagittal_index in range(imgs.shape[2]):
        sagittal = imgs[:, :, sagittal_index]
        sagittal = Image.fromarray(np.uint8(sagittal * 255) , 'L')

        sagittal = sagittal.resize([img_width, img_height])
        sagittal.save(os.path.join(patient_sagittal_dir, f"{sagittal_index}.png"))

In [9]:
def save_coronal(imgs, patient_coronal_dir, aspect):
    img_width = imgs.shape[2]
    # 그냥 int 를 취했다 round 하지 않았다.
    img_height = int(imgs.shape[0] * aspect)

    for coronal_index in range(imgs.shape[1]):
        coronal = imgs[:, coronal_index, :]
        coronal = Image.fromarray(np.uint8(coronal * 255) , 'L')

        coronal = coronal.resize([img_width, img_height])
        coronal.save(os.path.join(patient_coronal_dir, f"{coronal_index}.png"))     

In [ ]:
def save_axial(imgs, patient_axial_dir, aspect=None):

    for axial_index in range(imgs.shape[0]):
        axial = imgs[axial_index, :, :]
        axial = Image.fromarray(np.uint8(axial * 255) , 'L')

        axial.save(os.path.join(patient_axial_dir, f"{axial_index}.png"))

In [ ]:
def save_vertical_image(UID):

    patient_sagittal_dir = os.path.join(TRAIN_SAGITTAL_DIR, UID)
    patient_coronal_dir = os.path.join(TRAIN_CORONAL_DIR, UID)
    patient_axial_dir = os.path.join(TRAIN_AXIAL_DIR, UID)

    os.mkdir(patient_sagittal_dir)
    os.mkdir(patient_coronal_dir)
    os.mkdir(patient_axial_dir)

    imgs, image_positions, image_orientations, pixel_spacings, slice_thicknesses, is_flip = read_patient_dcm(os.path.join(TRAIN_IMAGE_DIR, UID))

    z_spacing = image_positions[0, 2] - image_positions[1, 2]
    pixel_spacing = pixel_spacings[0, 0]
    aspect = z_spacing / pixel_spacing # ㅇㅕ기를 원래 round 했다. repeat 하길래서 round 시킨걸 계속 갖고 있었네 ㅜㅜ
    imgs = normalize_hu(imgs)

    save_sagittal(imgs, patient_sagittal_dir, aspect)
    save_coronal(imgs, patient_coronal_dir, aspect)
    save_axial(imgs, patient_axial_dir, aspect)
        
    write_result = writer.writerow([
        UID,
        z_spacing, pixel_spacing, aspect, int(is_flip), imgs.shape[0]
    ])

    return write_result

In [ ]:
patient_dirs_iter = tqdm(patient_dirs)
for patient_dir in patient_dirs_iter:
    UID = patient_dir.split("/")[-1]
    
    save_vertical_image(UID)

    patient_dirs_iter.set_description(UID)


In [ ]:
f.close()